This will plot and compare the resulting data. To run this, make sure that you have already generated all data you want to be compared. If you want to run a sequential sort or the multiprocessing merge sort, run with run.py. If you want to run the MPI merge sort, run with mpi_run.py. 

In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from cloudmesh.common.util import banner
from analysis import get_data
from analysis import read_log, read_logs
from analysis import generate_average

In [11]:
user = "alex"
node = "v100"
sorts = ["seq-merge", "mp-mergesort"]
sort = "mp-mergesort"

sizes = [100, 1000, 1e4, 1e5, 1e6, 1e7, 1e8]
sizes = [int(size) for size in sizes]

In [ ]:
# this will take a long time. 
# also if you don't want to use all your processors then use different commands. 
for sort in sorts:
    for size in sizes:
        run_cmd = f"./run.py --user={user} --node={node} --size={size} --sort={sort}"
        banner(run_cmd)
        os.system(run_cmd)

In [3]:
def plot_benchmark_by_size(df, size=None, tag=None, files=None, x="sizes", y="time"):
    names = "-".join(files)
    sns.lineplot(data=df, x=x, y=y, hue="name");
    t = tag.title()
    label = f"{t}".replace("_", " ")
    plt.title(f"{label}")
    if "time" in y: y = f"{y}/s"
    if "time" in x: x = f"{x}/s"
    plt.xlabel(x.capitalize())
    plt.ylabel(y.capitalize())
    plt.savefig(f"images/{tag}-{names}.png")
    plt.savefig(f"images/{tag}-{names}.pdf")

In [12]:
files = ["alex"]
sorts = ["seq-merge", "mp-mergesort"]

frames = []
for file in files:
    for sort in sorts:
        frame = []
        for size in sizes: 
            size = int(size)
            log = f'{sort}-{node}-{file}'
            _frame = read_log(log, size=size, tag=sort)
            frame = frame + _frame
        frames.append(frame)
# print(frames)


df = pd.DataFrame()
for frame in frames:
    _df = pd.DataFrame(data=frame,
                columns=["processes", "time", "size", "name", "tag"])
    df = pd.concat([df, _df], ignore_index=True)
# plot_benchmark_by_size(df, "name", tag=sort, files=files, x="size", y="time")

In [13]:
df

,processes,time,size,name,tag
0,1,0.000,100,alex,seq-merge
1,1,0.000,100,alex,seq-merge
2,1,0.000,100,alex,seq-merge
3,1,0.000,100,alex,seq-merge
4,1,0.000,100,alex,seq-merge
...,...,...,...,...,...
2465,1,88.554,10000000,alex,mp-mergesort
2466,1,86.471,10000000,alex,mp-mergesort
2467,1,88.362,10000000,alex,mp-mergesort
2468,1,88.557,10000000,alex,mp-mergesort


In [14]:
df = df.pivot_table(
    values='time', index=['tag', 'processes'], columns=['name', 'size'], fill_value=0, aggfunc='mean')

In [15]:
df

name                        alex                                          \
size                   100       1000      10000     100000    1000000     
tag          processes                                                     
mp-mergesort 1            0.0309    0.0309    0.0443    0.4661    5.3893   
             2            0.0336    0.0380    0.0703    0.4890    5.6157   
             3            0.0380    0.0425    0.0780    0.5209    5.8003   
             4            0.0434    0.0465    0.0926    0.5345    5.8381   
             5            0.0486    0.0517    0.0988    0.5598    6.0120   
             6            0.0565    0.0591    0.1060    0.5535    6.1360   
             7            0.0612    0.0666    0.1133    0.5694    6.2229   
             8            0.0680    0.0733    0.1200    0.5705    6.2078   
             9            0.0745    0.0789    0.1289    0.6119    6.4212   
             10           0.0801    0.0840    0.1300    0.6140    6.2710   
             11           0.0868    0.0898    0.1362    0.6179    6.2842   
             12           0.0928    0.0958    0.1432    0.6192    6.3402   
             13           0.0972    0.1023    0.1481    0.6222    6.4917   
             14           0.1042    0.1076    0.1533    0.6308    6.4931   
             15           0.1105    0.1130    0.1615    0.6426    6.4822   
             16           0.1158    0.1200    0.1680    0.6531    6.3760   
             17           0.1200    0.1254    0.1747    0.6953    6.6491   
             18           0.1281    0.1323    0.1799    0.6975    6.5759   
             19           0.1332    0.1366    0.1859    0.6971    6.5626   
             20           0.1380    0.1418    0.1934    0.7024    6.5511   
             21           0.1442    0.1482    0.1972    0.7115    6.6155   
             22           0.1505    0.1542    0.2031    0.7176    6.6357   
             23           0.1545    0.1594    0.2088    0.7223    6.6533   
             24           0.1615    0.1638    0.2147    0.7258    6.6604   
             25           0.1677    0.1715    0.2197    0.7400    6.7546   
             26           0.1739    0.1784    0.2258    0.7391    6.7123   
             27           0.1800    0.1822    0.2319    0.7459    6.6935   
             28           0.1847    0.1875    0.2387    0.7516    6.6730   
             29           0.1914    0.1947    0.2458    0.7636    6.7195   
             30           0.1969    0.2006    0.2496    0.7652    6.6758   
             31           0.2026    0.2066    0.2569    0.7751    6.6061   
             32           0.2089    0.2116    0.2624    0.7785    6.6088   
             33           0.2145    0.2184    0.2716    0.8362    6.9463   
             34           0.2186    0.2227    0.2749    0.8310    6.8380   
             35           0.2257    0.2299    0.2822    0.8361    6.7757   
             36           0.2321    0.2360    0.2878    0.8422    6.7660   
             37           0.2379    0.2430    0.2922    0.8426    6.8370   
             38           0.2435    0.2457    0.2975    0.8477    6.8748   
             39           0.2471    0.2527    0.3035    0.8521    6.8792   
             40           1.8966    0.3555    0.4088    0.9468    6.9847   
seq-merge    1            0.0000    0.0026    0.0332    0.4278    5.3687   

name                                        
size                   10000000  100000000  
tag          processes                      
mp-mergesort 1           88.0766     0.000  
             2           90.2118     0.000  
             3           91.4093     0.000  
             4           83.3704     0.000  
             5           84.7213     0.000  
             6           84.3268     0.000  
             7           84.4062     0.000  
             8           84.9146     0.000  
             9           82.5112     0.000  
             10          81.6843     0.000  
             11          82.5078     0.000  
             12          

In [10]:
print(df.loc['mp-mergesort'])
speedup = df.rdiv(df.loc['seq-merge'].iloc[0])
speedup

name           alex                                                    \
size      100       1000      10000     100000    1000000   10000000    
processes                                                               
1            0.0309    0.0309    0.0443    0.4661    5.3893   88.0766   
2            0.0336    0.0380    0.0703    0.4890    5.6157   90.2118   
3            0.0380    0.0425    0.0780    0.5209    5.8003   91.4093   
4            0.0434    0.0465    0.0926    0.5345    5.8381   83.3704   
5            0.0486    0.0517    0.0988    0.5598    6.0120   84.7213   
6            0.0565    0.0591    0.1060    0.5535    6.1360   84.3268   
7            0.0612    0.0666    0.1133    0.5694    6.2229   84.4062   
8            0.0680    0.0733    0.1200    0.5705    6.2078   84.9146   
9            0.0745    0.0789    0.1289    0.6119    6.4212   82.5112   
10           0.0801    0.0840    0.1300    0.6140    6.2710   81.6843   
11           0.0868    0.0898    0.1362    0.6179  

name                        alex                                          \
size                   100       1000      10000     100000    1000000     
tag          processes                                                     
mp-mergesort 1               0.0  0.084142  0.749436  0.917829  0.996178   
             2               0.0  0.068421  0.472262  0.874847  0.956016   
             3               0.0  0.061176  0.425641  0.821271  0.925590   
             4               0.0  0.055914  0.358531  0.800374  0.919597   
             5               0.0  0.050290  0.336032  0.764202  0.892997   
             6               0.0  0.043993  0.313208  0.772900  0.874951   
             7               0.0  0.039039  0.293027  0.751317  0.862733   
             8               0.0  0.035471  0.276667  0.749869  0.864831   
             9               0.0  0.032953  0.257564  0.699134  0.836090   
             10              0.0  0.030952  0.255385  0.696743  0.856115   
             11              0.0  0.028953  0.243759  0.692345  0.854317   
             12              0.0  0.027140  0.231844  0.690891  0.846771   
             13              0.0  0.025415  0.224173  0.687560  0.827010   
             14              0.0  0.024164  0.216569  0.678186  0.826832   
             15              0.0  0.023009  0.205573  0.665733  0.828222   
             16              0.0  0.021667  0.197619  0.655030  0.842017   
             17              0.0  0.020734  0.190040  0.615274  0.807433   
             18              0.0  0.019652  0.184547  0.613333  0.816421   
             19              0.0  0.019034  0.178591  0.613685  0.818075   
             20              0.0  0.018336  0.171665  0.609055  0.819511   
             21              0.0  0.017544  0.168357  0.601265  0.811534   
             22              0.0  0.016861  0.163466  0.596154  0.809063   
             23              0.0  0.016311  0.159004  0.592275  0.806923   
             24              0.0  0.015873  0.154634  0.589419  0.806063   
             25              0.0  0.015160  0.151115  0.578108  0.794821   
             26              0.0  0.014574  0.147033  0.578812  0.799830   
             27              0.0  0.014270  0.143165  0.573535  0.802077   
             28              0.0  0.013867  0.139087  0.569186  0.804541   
             29              0.0  0.013354  0.135069  0.560241  0.798973   
             30              0.0  0.012961  0.133013  0.559070  0.804203   
             31              0.0  0.012585  0.129233  0.551929  0.812688   
             32              0.0  0.012287  0.126524  0.549518  0.812356   
             33              0.0  0.011905  0.122239  0.511600  0.772886   
             34              0.0  0.011675  0.120771  0.514801  0.785127   
             35              0.0  0.011309  0.117647  0.511661  0.792346   
             36              0.0  0.011017  0.115358  0.507955  0.793482   
             37              0.0  0.010700  0.113621  0.507714  0.785242   
             38              0.0  0.010582  0.111597  0.504660  0.780925   
             39              0.0  0.010289  0.109390  0.502054  0.780425   
             40              0.0  0.007314  0.081213  0.451838  0.768637   
seq-merge    1               NaN  1.000000  1.000000  1.000000  1.000000   

name                                        
size                   10000000  100000000  
tag          processes                      
mp-mergesort 1          0.790153       inf  
             2          0.771451       inf  
             3          0.761345       inf  
             4          0.834757       inf  
             5          0.821446       inf  
             6          0.825289       inf  
             7          0.824513       inf  
             8          0.819576       inf  
             9          0.843449       inf  
             10         0.851987       inf  
             11         0.843484       inf  
             12         0